In [1]:
# run this one
from pathlib import Path
import pytorch_lightning as pl
import subprocess
from mGPT.data.build_data import build_data
from mGPT.models.build_model import build_model
import torch
from huggingface_hub import snapshot_download
from omegaconf import OmegaConf

def load_cfg_from_file(file_path):
    """
    Load the configuration object from a file.

    Args:
    file_path (str): The path of the file from which to load the configuration.

    Returns:
    DictConfig: The loaded configuration object.
    """
    cfg = OmegaConf.load(file_path)
    return cfg
    
cfg = load_cfg_from_file('./config_mix.yaml')

cfg.FOLDER = 'cache'
output_dir = Path(cfg.FOLDER)
output_dir.mkdir(parents=True, exist_ok=True)
pl.seed_everything(cfg.SEED_VALUE)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model_path = snapshot_download(repo_id="bill-jiang/MotionGPT-base")

datamodule = build_data(cfg, phase="test")
model = build_model(cfg, datamodule)
state_dict = torch.load(f'{model_path}/motiongpt_s3_h3d.tar',
                        map_location="cpu")["state_dict"]
model.load_state_dict(state_dict)
model.to(device)

#input = data_stored[-1]['user_input']
motion_uploaded = {
        "feats": None,
        "joints": None,
        "motion_video": None,
        "motion_lengths": 0,
        "motion_token": None,
        "motion_token_string": '',
        "motion_token_length": 0,
    }
motion_length, motion_token_string = motion_uploaded[
        "motion_lengths"], motion_uploaded["motion_token_string"]

input = "right feet high kick then wave both hands"
prompt = model.lm.placeholder_fulfill(input, motion_length,motion_token_string, "")
batch = {
        "length": [motion_length],
        "text": [prompt],
}

outputs = model(batch, task="t2m")

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
Seed set to 1234


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

['right feet high kick then wave both hands']
['<motion_id_512><motion_id_295><motion_id_114><motion_id_245><motion_id_114><motion_id_114><motion_id_245><motion_id_114><motion_id_114><motion_id_379><motion_id_379><motion_id_9><motion_id_114><motion_id_114><motion_id_114><motion_id_114><motion_id_114><motion_id_114><motion_id_114><motion_id_245><motion_id_9><motion_id_458><motion_id_114><motion_id_114><motion_id_379><motion_id_379><motion_id_379><motion_id_114><motion_id_114><motion_id_379><motion_id_379><motion_id_114><motion_id_226><motion_id_379><motion_id_149><motion_id_124><motion_id_129><motion_id_295><motion_id_319><motion_id_379><motion_id_114><motion_id_114><motion_id_114><motion_id_114><motion_id_114><motion_id_347><motion_id_347><motion_id_379><motion_id_114><motion_id_458><motion_id_513>']


In [2]:
outputs

{'texts': ['<Motion_Placeholder>'],
 'feats': tensor([[[  1.9248,   0.9576,  -6.4742,  ...,   9.8800, -59.8608, -61.1447],
          [  2.4321,   3.2078, -10.0685,  ...,   9.8921, -59.2283, -60.0052],
          [  2.2217,   2.5871, -15.4734,  ...,   9.7138, -60.0113, -61.0589],
          ...,
          [  0.8327,   0.1854, -14.5749,  ...,   9.7644,  10.6365,   9.6429],
          [  1.5885,  -0.5432, -13.6270,  ...,   9.9343,  10.6977,   9.9476],
          [  1.2215,   1.6911, -15.0894,  ...,   9.9263,  10.8049,   9.8728]]],
        device='cuda:0'),
 'joints': tensor([[[[ 0.0000e+00,  9.8100e-01,  0.0000e+00],
           [ 3.0556e-02,  8.8654e-01,  1.2800e-02],
           [-6.2775e-02,  9.3311e-01,  7.3377e-02],
           ...,
           [-2.4270e-01,  1.1528e+00,  6.0705e-02],
           [ 2.0463e-01,  1.1551e+00,  3.0819e-01],
           [-1.3897e-01,  1.1578e+00,  3.2604e-01]],
 
          [[ 6.1566e-04,  9.7774e-01,  4.3032e-03],
           [ 3.7207e-02,  8.8402e-01,  1.4990e-02],

In [ ]:
import requests, json, numpy

# The URL to your endpoint
url = "http://localhost:9880/generate_motion"

# The data to send in the request, replace this with your actual data
data = {
    "text": "example text prompt"
}

# Send a POST request
response = requests.post(url, json=data)

# Check if the request was successful
if response.status_code == 200:
    print("Request was successful.")
    print("Response data:", response.json())
else:
    print("Request failed.")
    print("Status code:", response.status_code)
    print("Response data:", response.text)


# Assuming 'response' is the variable containing the byte string from the server
response_data = response.content.decode('utf-8')  # Step 1: Decode
data = json.loads(response_data)  # Step 2: Parse

# Now, 'data' is a Python dictionary containing the response.
# You can access the 'joints' and 'length' like so:
joints = data['joints']
length = data['length']

In [114]:
def render_motion(data, feats, method='fast'):
    fname = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime(
        time.time())) + str(np.random.randint(10000, 99999))
    video_fname = fname + '.mp4'
    feats_fname = f"{fname}_feats" + '.npy'
    data_fname = f"{fname}_joints" + '.npy'
    output_npy_path = os.path.join(output_dir, feats_fname)
    output_joints_path = os.path.join(output_dir, data_fname)
    output_mp4_path = os.path.join(output_dir, video_fname)
    np.save(output_npy_path, feats)
    np.save(output_joints_path, data)
    
    render_cmd = ["python", "-m", "render", "--joints_path", output_joints_path, "--method", method, "--output_mp4_path", output_mp4_path, "--smpl_model_path", cfg.RENDER.SMPL_MODEL_PATH]
    os.system(" ".join(render_cmd))
    # subprocess.run(cmd3)
    
    return output_mp4_path, video_fname, output_npy_path, feats_fname
    

In [115]:
#method = either["slow", "fast"]
method = 'slow'

In [116]:
out_lengths = outputs["length"][0]
out_lengths

188

In [117]:
out_joints = outputs["joints"][:out_lengths].detach().cpu().numpy()

In [124]:
# 将 ndarray 转换为 list
outputs_joints_list = outputs["joints"][:out_lengths].tolist()

# 将 list 保存为 JSON 文件
with open('mgpt_outputs_joints_list.json', 'w') as f:
    json.dump(outputs_joints_list, f)

In [118]:
import time
import numpy as np
import os
render_motion(
        out_joints,
        outputs["feats"].to('cpu').numpy(), method)

Moviepy - Building video cache/2024-01-14-02_41_1628163.mp4.
Moviepy - Writing video cache/2024-01-14-02_41_1628163.mp4



Moviepy - Done !
Moviepy - video ready cache/2024-01-14-02_41_1628163.mp4


('cache/2024-01-14-02_41_1628163.mp4',
 '2024-01-14-02_41_1628163.mp4',
 'cache/2024-01-14-02_41_1628163_feats.npy',
 '2024-01-14-02_41_1628163_feats.npy')

In [125]:
out_joints.shape

(1, 188, 22, 3)

In [126]:
import OpenGL.GL as gl
import imageio
import cv2
import random
import torch
import moviepy.editor as mp
from scipy.spatial.transform import Rotation as RRR
import mGPT.render.matplot.plot_3d_global as plot_3d
from mGPT.render.pyrender.hybrik_loc2rot import HybrIKJointsToRotmat
from mGPT.render.pyrender.smpl_render import SMPLRender

In [127]:
data = out_joints.copy()

In [128]:
        if len(data.shape) == 4:
            data = data[0]


In [130]:
data = data - data[0,0]

In [131]:
data.shape

(188, 22, 3)

In [132]:
pose_generator = HybrIKJointsToRotmat()

In [133]:
pose = pose_generator(data)

In [137]:

# 将 ndarray 转换为 list
outputs_pose_rot_matrix_list = pose.tolist()

# 将 list 保存为 JSON 文件
with open('mgpt_outputs_pose_rot_matrix_list.json', 'w') as f:
    json.dump(outputs_pose_rot_matrix_list, f)

In [134]:
pose.shape

(188, 22, 3, 3)

In [135]:
pose[0][0]

array([[ 0.84523808,  0.02547463,  0.53378239],
       [-0.34008805,  0.7961213 ,  0.5005307 ],
       [-0.41220469, -0.60460062,  0.68157566]])

In [60]:
# for finger tips, 
# np.stack([np.stack([np.eye(3)] * pose.shape[0], 0)] * 2, 1).shape

(168, 2, 3, 3)

In [62]:
#给root rotation 在x轴上翻转180度
# r = RRR.from_rotvec(np.array([np.pi, 0.0, 0.0]))
# pose[:, 0] = np.matmul(r.as_matrix().reshape(1, 3, 3), pose[:, 0])

(168, 3, 3)

In [136]:
#各个joint的老母
parents = [0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 12, 13, 14, 16, 17, 18, 19]

In [65]:
data[:,parents].shape

(168, 22, 3)

In [ ]:
#每个减去老母（如果有老母）的位置
#joints_rel = joints - joints[:, self.parents]
#->data = data - data[:, parents]

In [138]:
pose

array([[[[ 8.45238077e-01,  2.54746348e-02,  5.33782386e-01],
         [-3.40088054e-01,  7.96121303e-01,  5.00530704e-01],
         [-4.12204692e-01, -6.04600623e-01,  6.81575659e-01]],

        [[ 9.56222762e-01, -2.73963064e-01,  1.02870158e-01],
         [ 2.91101238e-01,  8.54490289e-01, -4.30239950e-01],
         [ 2.99683041e-02,  4.41350864e-01,  8.96834051e-01]],

        [[ 8.66053509e-01,  3.15664930e-01,  3.87694430e-01],
         [-4.62149590e-01,  2.09671855e-01,  8.61658557e-01],
         [ 1.90706778e-01, -9.25415239e-01,  3.27471464e-01]],

        ...,

        [[-6.29496691e-01,  2.35637953e-01,  7.40411150e-01],
         [-1.25278460e-01,  9.09658704e-01, -3.96013068e-01],
         [-7.66837155e-01, -3.42046484e-01, -5.43106785e-01]],

        [[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

        [[ 1.00000000e+00,  0.00000000e+00,  0

In [ ]:
# righthip-2 test

In [139]:
rh_0 = pose[0,2]
rh_0

array([[ 0.86605351,  0.31566493,  0.38769443],
       [-0.46214959,  0.20967185,  0.86165856],
       [ 0.19070678, -0.92541524,  0.32747146]])

In [140]:
rh_1 = pose[1,2]
rh_1

array([[ 0.80091957,  0.40609643,  0.44001538],
       [-0.54555257,  0.19204762,  0.81577577],
       [ 0.24677972, -0.8934223 ,  0.37536165]])

In [141]:
from scipy.spatial.transform import Rotation as R

# 将旋转矩阵转换为四元数
rh_0_rotation = R.from_matrix(rh_0)
rh_0_quaternion = rh_0_rotation.as_quat()

In [143]:
rh_1_rotation = R.from_matrix(rh_1)
rh_1_quaternion = rh_1_rotation.as_quat()

In [142]:
#xyzw
rh_0_quaternion

array([-0.57639184,  0.06353519, -0.25087153,  0.77511238])

In [144]:
rh_1_quaternion

array([-0.55531755,  0.06278216, -0.30919026,  0.76946878])

In [79]:
!pip install pyrr

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [80]:
import pyrr

In [145]:
rh_0_quaternion_inv = pyrr.quaternion.inverse(rh_0_quaternion)
rh_0_quaternion_inv

array([ 0.57639184, -0.06353519,  0.25087153,  0.77511238])

In [146]:
rh_0_quaternion

array([-0.57639184,  0.06353519, -0.25087153,  0.77511238])

In [147]:
#verified
def quaternion_inverse_xyzw(quaternion):
    """
    计算以 [x, y, z, w] 形式表示的四元数的逆。
    :param quaternion: 四元数，格式为 [x, y, z, w]
    :return: 四元数的逆
    """
    # 提取四元数的各个分量
    x, y, z, w = quaternion

    # 计算四元数的范数的平方
    norm_squared = x * x + y * y + z * z + w * w

    # 计算逆四元数
    inverse_quaternion = np.array([-x, -y, -z, w]) / norm_squared

    return inverse_quaternion
rh_0_quaternion_inv = quaternion_inverse_xyzw(rh_0_quaternion)
rh_0_quaternion_inv

array([ 0.57639184, -0.06353519,  0.25087153,  0.77511238])

In [148]:
#verified
def multiply_quaternions(q1, q2):
    """
    Multiply two quaternions.
    Quaternion format: [x, y, z, w]

    :param q1: Quaternion 1 as [x, y, z, w]
    :param q2: Quaternion 2 as [x, y, z, w]
    :return: Resultant quaternion of multiplication
    """
    x1, y1, z1, w1 = q1
    x2, y2, z2, w2 = q2

    qx = w1*x2 + x1*w2 + y1*z2 - z1*y2
    qy = w1*y2 - x1*z2 + y1*w2 + z1*x2
    qz = w1*z2 + x1*y2 - y1*x2 + z1*w2
    qw = w1*w2 - x1*x2 - y1*y2 - z1*z2

    return [qx, qy, qz, qw]

result = multiply_quaternions(rh_1_quaternion, rh_0_quaternion_inv)
print("Result of quaternion multiplication:", result)

Result of quaternion multiplication: [0.009187813789874138, -0.039126491774203104, -0.04752430535941932, 0.9980611915705924]


In [150]:
pyrr.quaternion.cross(rh_1_quaternion, rh_0_quaternion_inv)

array([ 0.00918781, -0.03912649, -0.04752431,  0.99806119])

In [95]:
#verified
def quaternion_to_unity_euler_xyz_wxyz(quaternion):
    # Ensure quaternion is in the format (w, x, y, z)
    w, x, y, z = quaternion
    # Create a Rotation object
    rotation = R.from_quat([x, y, z, w])
    # Convert to Euler angles in radians
    euler_radians = rotation.as_euler('xyz', degrees=False)
    # Convert radians to degrees, as Unity uses degrees
    euler_degrees = np.degrees(euler_radians)
    return euler_degrees

In [96]:
quaternion_to_unity_euler_xyz_wxyz([ 0.00888,-0.00012, -0.01798, 0.99980])

array([-2.06050588e+00, -4.54771650e-03,  1.78982332e+02])

In [100]:
1.78982332e+02

178.982332

In [151]:
# verified
from scipy.spatial.transform import Rotation as R

# Assuming pose is your input array with shape (n, 22, 3, 3)

# Validate the shape of the array
if pose.shape[1:] != (22, 3, 3):
    raise ValueError("Invalid shape of the input array. Expected (n, 22, 3, 3)")

# Initialize the output array
n = pose.shape[0]
quaternions = np.empty((n, 22, 4))

# Convert each rotation matrix to a quaternion
for i in range(n):
    for j in range(22):
        rotation_matrix = pose[i, j]
        quaternion = R.from_matrix(rotation_matrix).as_quat()
        quaternions[i, j] = quaternion

In [155]:
quaternions[1][2]

array([-0.55531755,  0.06278216, -0.30919026,  0.76946878])

In [156]:
# 将 ndarray 转换为 list
outputs_quaternions_list = quaternions.tolist()

# 将 list 保存为 JSON 文件
with open('mgpt_outputs_quaternions_list.json', 'w') as f:
    json.dump(outputs_quaternions_list, f)

In [159]:
def calculate_rot_diff(array):
    n, num_joints, _ = array.shape
    rot_diff = np.zeros((n-1, num_joints, 4))

    for i in range(n-1):
        for j in range(num_joints):
            quat_prev = array[i, j]
            quat_curr = array[i+1, j]
            quat_prev_inv = quaternion_inverse_xyzw(quat_prev)
            rot_diff[i, j] = multiply_quaternions(quat_curr, quat_prev_inv)

    return rot_diff

In [160]:
quat_diff = calculate_rot_diff(quaternions)

In [162]:
quat_diff.shape

(187, 22, 4)

In [163]:
quat_diff[0][2]

array([ 0.00918781, -0.03912649, -0.04752431,  0.99806119])